In [1]:
import numpy as np

def generate_training_data(num_samples=105, c=4):
    X_train = []
    y_train = []

    # Define probabilities for winding numbers
    probabilities = [0.13, 0.17, 0.20, 0.32, 0.18]
    for _ in range(num_samples):
        # Generate random coefficients for Fourier series
        #[-10,10]
        a_x = np.random.uniform(-1, 1, size=c+1)
        b_x = np.random.uniform(-1, 1, size=c+1)
        a_y = np.random.uniform(-1, 1, size=c+1)
        b_y = np.random.uniform(-1, 1, size=c+1)

        # Generate winding number
        w = np.random.choice([-2, -1, 0, 1, 2], p=probabilities)

        # Append Hamiltonian and winding number to training data
        X_train.append((a_x, b_x, a_y, b_y))
        y_train.append(w)

    return np.array(X_train), np.array(y_train)

# Example usage
X_train, y_train = generate_training_data(num_samples=1000)

In [2]:
X_train.shape, y_train.shape

((1000, 4, 5), (1000,))

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Define the fully connected network architecture
model = Sequential([
    Dense(40, activation='relu', input_shape=(20,)),  # First hidden layer with 40 neurons
    Dense(32, activation='relu'),  # Second hidden layer with 32 neurons
    Dense(2, activation='linear')  # Output layer with 2 neurons
])

# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                840       
                                                                 
 dense_1 (Dense)             (None, 32)                1312      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 2218 (8.66 KB)
Trainable params: 2218 (8.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split


# Clear the previous TensorFlow session
tf.keras.backend.clear_session()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train_flat = np.array([sample.flatten().reshape(-1) for sample in X_train])

X_test_flat = np.array([sample.flatten().reshape(-1) for sample in X_test])

# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_flat, y_train, epochs=50, batch_size=32, validation_data=(X_test_flat, y_test))

Epoch 1/50
25/25 [==============================] - 3s 44ms/step - loss: 1.8622 - accuracy: 0.2350 - val_loss: 1.8129 - val_accuracy: 0.2650
Epoch 2/50
25/25 [==============================] - 0s 20ms/step - loss: 1.7627 - accuracy: 0.2412 - val_loss: 1.7981 - val_accuracy: 0.2600
Epoch 3/50
25/25 [==============================] - 0s 18ms/step - loss: 1.7250 - accuracy: 0.2425 - val_loss: 1.7982 - val_accuracy: 0.2800
Epoch 4/50
25/25 [==============================] - 0s 18ms/step - loss: 1.6995 - accuracy: 0.2450 - val_loss: 1.7992 - val_accuracy: 0.2950
Epoch 5/50
25/25 [==============================] - 0s 15ms/step - loss: 1.6823 - accuracy: 0.2475 - val_loss: 1.8058 - val_accuracy: 0.2850
Epoch 6/50
25/25 [==============================] - 1s 26ms/step - loss: 1.6567 - accuracy: 0.2463 - val_loss: 1.8105 - val_accuracy: 0.2850
Epoch 7/50
25/25 [==============================] - 1s 26ms/step - loss: 1.6369 - accuracy: 0.2450 - val_loss: 1.8136 - val_accuracy: 0.2750
Epoch 8/50
25

In [5]:
# Evaluate the model on the training data
loss, accuracy = model.evaluate(X_train_flat, y_train)
print("Training Loss:", loss)
print("Training Accuracy:", accuracy)

# Evaluate the model on the test data (assuming you have X_test_flat and y_test)
loss, accuracy = model.evaluate(X_test_flat, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


25/25 [==============================] - 0s 2ms/step - loss: 0.6431 - accuracy: 0.2325
Training Loss: 0.6431195735931396
Training Accuracy: 0.23250000178813934
7/7 [==============================] - 0s 3ms/step - loss: 2.5446 - accuracy: 0.2250
Test Loss: 2.5445759296417236
Test Accuracy: 0.22499999403953552


In [11]:
!pip install scikeras

In [17]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.layers import Dense, Dropout

c=4
# Define a function to create the model
def create_model(dropout_rate=0.2):
    model = Sequential([
        Dense(40, activation='relu', input_shape=(4*(c+1),)),
        Dropout(dropout_rate),
        Dense(32, activation='relu'),
        Dropout(dropout_rate),
        Dense(2, activation='linear')
    ])
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier based on the model function
model = KerasClassifier(build_fn=create_model, dropout_rate=0.2, verbose=0)

# Define the parameter grid
param_grid = {
    'batch_size': [16, 32, 64],
    'epochs': [50, 100, 150],
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3, 0.4]
}

# Create a GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Perform grid search
grid_result = grid.fit(X_train_flat, y_train)

# Print the best parameters and best accuracy
print("Best Parameters:", grid_result.best_params_)
print("Best Accuracy:", grid_result.best_score_)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras

Best Parameters: {'batch_size': 64, 'dropout_rate': 0.4, 'epochs': 100, 'optimizer': 'rmsprop'}
Best Accuracy: 0.17500680540302063
